In [1]:
%load_ext autoreload
%autoreload 1
%aimport my

import sys
import numpy as np
import pandas as pd

import os
import gc
import matplotlib.pyplot as plt
import importlib
import pickle

import my
from my import p
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 200)
pd.set_option("max_colwidth", 45)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from sklearn.model_selection import train_test_split

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)
     
dir_out = "out/"
dir_data = 'data/'

SEED = 34
np.random.seed(SEED)
N_CPU = os.cpu_count()

imports

In [2]:
pq_dir = 'data/d/'
pq_files = sorted(os.listdir(path=pq_dir))
pq_files

['part0.pq',
 'part1.pq',
 'part2.pq',
 'part3.pq',
 'part4.pq',
 'part5.pq',
 'part6.pq',
 'part7.pq',
 'part8.pq',
 'part9.pq']

In [ ]:
# for i,name in enumerate(pq_files):
#     os.rename(pq_dir+name, pq_dir+f'part{i}.pq')

Объединение данных

In [3]:
df = []
cat_cols = ['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name','url_host', 'cpe_type_cd', 'cpe_model_os_type', 'price']

for name in pq_files:
    q = pd.read_parquet(pq_dir+name)
    q['user_id'] = q['user_id'].astype(np.int32) 
    q['price'] = q['price'].astype(np.float32) 
    q['date'] = pd.to_datetime(q['date'])
    q['part_of_day'] = q['part_of_day'].map(my.TIMEtoINT).astype(np.int8)
    q['request_cnt'] = q['request_cnt'].astype(np.int8)

    for col in cat_cols:
        q[col] = q[col].astype('category')

    df.append(q)
    del q
    gc.collect()

df = pd.concat(df,ignore_index=True)

df['price'] = df['price'].astype(np.float32)
for col in cat_cols:
    df[col] = df[col].astype('category')

gc.collect()
p(df.shape)
df[:3]

(322899435, 12)


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,20368.000,2022-06-15,1,1,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,20368.000,2022-06-19,1,1,45098
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,20368.000,2022-06-12,2,1,45098


In [4]:
df.dtypes

region_name                    category
city_name                      category
cpe_manufacturer_name          category
cpe_model_name                 category
url_host                       category
cpe_type_cd                    category
cpe_model_os_type              category
price                          category
date                     datetime64[ns]
part_of_day                        int8
request_cnt                        int8
user_id                           int32
dtype: object

In [5]:
df['user_id'].nunique()

415317

In [6]:
df.to_parquet('out/pre/df_all.pq', index=False)

In [3]:
df = pd.read_parquet('out/pre/df_all.pq')
df[:2]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,20368.000,2022-06-15,1,1,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,20368.000,2022-06-19,1,1,45098


Распределение цен телефонов по бинам, заполнение nan

In [7]:
def mean_price(x):
    uniq_ps = x.unique()
    if (len(uniq_ps) == 1) and np.isnan(uniq_ps[0]):
        return -1
    return np.nanmean(uniq_ps).item()

df['price'] = df.groupby('cpe_model_name')['price'].transform(mean_price).astype(np.float32)

In [8]:
price_bins = [-np.inf,0,2000,5000,10000,15000,20000,25000, 35000,50000,70000, 100000,np.inf]
price_labels = list(range(1,len(price_bins)))
df['price'] = pd.cut(df['price'],bins=price_bins, labels=price_labels)
df['price'].dtype

CategoricalDtype(categories=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ordered=True)

In [9]:
df['price'].value_counts().sort_index()

1      3482705
2         2309
3       743018
4     28821130
5     47248161
6     52807804
7     35535781
8     24812580
9     33378542
10    61915653
11    25509481
12     8642271
Name: price, dtype: int64

In [10]:
df[:3]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,9,2022-06-15,1,1,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,9,2022-06-19,1,1,45098
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,9,2022-06-12,2,1,45098


In [11]:
df.to_parquet('out/pre/df_all_1.pq', index=False)

In [ ]:
df = pd.read_parquet('out/pre/df_all_1.pq')

In [12]:
df['part_of_day'].unique()

array([1, 2, 3, 4], dtype=int8)

In [13]:
df['date'] = df['date'] + pd.to_timedelta(df['part_of_day'],unit='hour')
df[:3]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,9,2022-06-15 01:00:00,1,1,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,9,2022-06-19 01:00:00,1,1,45098
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,9,2022-06-12 02:00:00,2,1,45098


In [14]:
df['request_cnt'] = (df['request_cnt'].clip(0,12)/12).astype(np.float32)
df[:2]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,9,2022-06-15 01:00:00,1,0.083,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,9,2022-06-19 01:00:00,1,0.083,45098


In [15]:
gc.collect()

0

In [16]:
df.to_parquet('out/pre/df_all_2.pq', index=False)

In [2]:
df = pd.read_parquet('out/pre/df_all_2.pq')
p(df.shape)
df[:2]

(322899435, 12)


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,5,2022-06-15 01:00:00,1,0.083,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,5,2022-06-19 01:00:00,1,0.083,45098


LabelEncoder для категориальных признаков

In [17]:
from sklearn.preprocessing import LabelEncoder

cols_to_encode = ['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name','url_host', 'cpe_type_cd', 'cpe_model_os_type']

# todo frequency encoder 
encoders = {}
for col in cols_to_encode:
    encoders[col] = LabelEncoder().fit(df[col])

my.save_pickle('out/encs.pik', encoders)

In [27]:
df[:2]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,9,2022-06-15 01:00:00,1,0.083,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,9,2022-06-19 01:00:00,1,0.083,45098


In [28]:
encoders

{'region_name': LabelEncoder(),
 'city_name': LabelEncoder(),
 'cpe_manufacturer_name': LabelEncoder(),
 'cpe_model_name': LabelEncoder(),
 'url_host': LabelEncoder(),
 'cpe_type_cd': LabelEncoder(),
 'cpe_model_os_type': LabelEncoder()}

In [29]:
for name, e in encoders.items():
    print(name, len(e.classes_))

region_name 81
city_name 985
cpe_manufacturer_name 37
cpe_model_name 599
url_host 199683
cpe_type_cd 4
cpe_model_os_type 3


In [30]:
df = my.label_encode(df, cols_to_encode, encoders)
gc.collect()
df[:3]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,22,410,2,590,5789,3,3,9,2022-06-15 01:00:00,1,0.083,45098
1,22,410,2,590,12901,3,3,9,2022-06-19 01:00:00,1,0.083,45098
2,22,410,2,590,17627,3,3,9,2022-06-12 02:00:00,2,0.083,45098


In [32]:
df.to_parquet('out/pre/df_all_3.pq', index=False)

Сортируем по дате

In [2]:
df = pd.read_parquet('out/pre/df_all_3.pq')
p(df.shape)
df[:2]

(322899435, 12)


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,22,410,2,590,5789,3,3,9,2022-06-15 01:00:00,1,0.083,45098
1,22,410,2,590,12901,3,3,9,2022-06-19 01:00:00,1,0.083,45098


In [3]:
df.sort_values(['user_id','date'], inplace=True)
gc.collect()

0

In [4]:
df.to_parquet('out/pre/df_all_4.pq', index=False)

TRAIN_VAL

In [2]:
df = pd.read_parquet('out/pre/df_all_4.pq')
df.shape

(322899435, 12)

In [3]:
train = pd.read_parquet('data/public_train.pqt').reset_index(drop=True)
train

,age,is_male,user_id
0,31.000,1,350459
1,35.000,1,188276
2,41.000,0,99002
3,33.000,0,155506
4,54.000,0,213873
...,...,...,...
269995,49.000,1,225374
269996,22.000,1,25776
269997,28.000,0,148131
269998,28.000,1,205570


In [4]:
X_train_val = df.loc[df['user_id'].isin(train['user_id'])]
gc.collect()
X_train_val.shape

(210730732, 12)

In [5]:
del df
gc.collect()

0

In [6]:
# должны быть отсортированы
# Number each item in each group from 0 to the length of that group - 1.
X_train_val['pos'] = X_train_val.groupby('user_id').cumcount(ascending=False).astype(np.int16)
X_train_val[:2]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,pos
0,29,531,28,156,117490,3,1,4,2021-06-16 02:00:00,2,0.083,0,130
1,29,531,28,156,111474,3,1,4,2021-06-16 02:00:00,2,0.083,0,129


In [7]:
X_train_val.shape

(210730732, 13)

In [8]:
X_train_val = X_train_val.loc[X_train_val['pos']<5000].sort_values(['user_id','date'])
X_train_val.shape

(206741456, 13)

In [9]:
X_train_val.to_parquet('out/pre/X_train_val_1.pq', index=False)

In [10]:
del X_train_val
gc.collect()

0

TEST

In [11]:
df = pd.read_parquet('out/pre/df_all_4.pq')

In [14]:
test = pd.read_parquet('data/submit_2.pqt').reset_index(drop=True)
test

,user_id
0,221301
1,31271
2,211594
3,253119
4,192578
...,...
144719,415284
144720,415285
144721,415286
144722,415306


In [15]:
X_test = df.loc[df['user_id'].isin(test['user_id'])]
gc.collect()
X_test.shape

(111734107, 12)

In [16]:
del df
gc.collect()

0

Оставляем максимум 5000 элементов в последовательностях

In [17]:
# должны быть отсортированы
# Number each item in each group from 0 to the length of that group - 1.
X_test['pos'] = X_test.groupby('user_id').cumcount(ascending=False).astype(np.int16)
p(X_test.shape)
X_test = X_test.loc[X_test['pos']<5000].sort_values(['user_id','date'])
p(X_test.shape)
X_test[:2]

(111734107, 13)
(109567390, 13)


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,pos
4485,30,239,14,274,59367,3,1,4,2021-06-16 01:00:00,1,0.167,6,455
4486,30,239,14,274,160868,3,1,4,2021-06-16 01:00:00,1,0.250,6,454


In [18]:
X_test.to_parquet('out/pre/X_test_1.pq', index=False)

In [19]:
del X_test
gc.collect()

0

In [17]:
# df['request_cnt'].value_counts()

1     201324004
2      64345547
3      26781698
4      14887615
5       8360497
6       6665296
8        266005
10       156054
12       111868
7           788
9            29
11           17
14           14
16            2
13            1
Name: request_cnt, dtype: int64

In [14]:
# pd.to_timedelta(df['part_of_day'].iloc[:5],unit='hour')

0   0 days 01:00:00
1   0 days 01:00:00
2   0 days 02:00:00
3   0 days 02:00:00
4   0 days 02:00:00
Name: part_of_day, dtype: timedelta64[ns]

In [15]:
# df['date'].iloc[:5] + pd.to_timedelta(df['part_of_day'].iloc[:5],unit='hour')

0   2022-06-15 01:00:00
1   2022-06-19 01:00:00
2   2022-06-12 02:00:00
3   2022-05-16 02:00:00
4   2022-05-30 02:00:00
dtype: datetime64[ns]

Наполняем вспомогательный датафрейм для таргетов

In [2]:
train = pd.read_parquet('data/public_train.pqt').reset_index(drop=True)
train

,age,is_male,user_id
0,31.000,1,350459
1,35.000,1,188276
2,41.000,0,99002
3,33.000,0,155506
4,54.000,0,213873
...,...,...,...
269995,49.000,1,225374
269996,22.000,1,25776
269997,28.000,0,148131
269998,28.000,1,205570


In [3]:
train.dtypes

age        float64
is_male     object
user_id      int64
dtype: object

In [4]:
train['is_male'].value_counts(dropna=False)

1       135332
0       128994
NA        5632
None        42
Name: is_male, dtype: int64

In [5]:
train['age'].value_counts(dropna=False)

34.000    10004
33.000     9948
35.000     9456
36.000     9401
37.000     9341
38.000     9106
32.000     9080
31.000     8979
30.000     8591
39.000     8481
29.000     8295
40.000     8228
41.000     7910
26.000     7799
28.000     7607
42.000     7571
27.000     7511
43.000     7488
25.000     7398
24.000     7133
44.000     7083
23.000     6787
22.000     5398
49.000     5180
48.000     4969
50.000     4841
51.000     4613
47.000     4549
52.000     4170
53.000     3888
54.000     3611
46.000     3385
55.000     3236
56.000     3128
57.000     3001
21.000     2995
58.000     2909
45.000     2877
59.000     2816
60.000     2574
61.000     2355
62.000     2048
63.000     1848
20.000     1553
64.000     1505
65.000     1396
19.000     1377
66.000     1207
67.000      998
68.000      829
69.000      751
18.000      726
70.000      598
71.000      428
72.000      269
16.000      130
17.000      125
74.000      101
73.000       92
15.000       53
75.000       43
76.000       37
77.000  

In [6]:
train = pd.read_parquet('data/public_train.pqt').reset_index(drop=True)
train['old_age'] = train['age'].fillna(-1).astype(np.int8)
train['is_male'] = train['is_male'].replace('NA',np.nan).astype(np.float32).fillna(-1).astype(np.int8)       

train['age'] = train['old_age'].apply(my.cat_age).astype(np.int8)
p(train.shape)
train.columns = ['target_age','target_is_male','user_id','old_age']
train[:5]

(270000, 4)


,target_age,target_is_male,user_id,old_age
0,1,1,350459,31
1,1,1,188276,35
2,2,0,99002,41
3,1,0,155506,33
4,3,0,213873,54


In [7]:
(train['target_age'] == -1).sum(), (train['target_is_male'] == -1).sum()

(1, 5674)

In [8]:
train.dtypes

target_age         int8
target_is_male     int8
user_id           int64
old_age            int8
dtype: object

In [9]:
train.isna().sum()

target_age        0
target_is_male    0
user_id           0
old_age           0
dtype: int64

In [10]:
X_train_val = pd.read_parquet('out/pre/X_train_val_1.pq')
p(X_train_val.shape)
X_train_val[:3]

(206741456, 13)


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,pos
0,29,531,28,156,117490,3,1,4,2021-06-16 02:00:00,2,0.083,0,130
1,29,531,28,156,111474,3,1,4,2021-06-16 02:00:00,2,0.083,0,129
2,29,531,28,156,59367,3,1,4,2021-06-16 03:00:00,3,0.083,0,128


In [11]:
user_lens = X_train_val['user_id'].value_counts().astype(np.int16)
user_lens

341574    5000
252426    5000
56287     5000
284198    5000
363674    5000
          ... 
149487       1
149488       1
151600       1
151614       1
415313       1
Name: user_id, Length: 270000, dtype: int16

In [12]:
tel_prices = X_train_val.groupby('user_id')['price'].max().astype(np.int16)
tel_prices

user_id
0         4
1         8
2         5
3         5
4         8
         ..
415308    4
415309    6
415311    4
415312    4
415313    4
Name: price, Length: 270000, dtype: int16

In [14]:
train['user_len'] = user_lens.reindex(train['user_id']).reset_index(drop=True)
train['tel_price'] = tel_prices.reindex(train['user_id']).reset_index(drop=True)
train[:3]

,target_age,target_is_male,user_id,old_age,user_len,tel_price
0,1,1,350459,31,178,5
1,1,1,188276,35,111,7
2,2,0,99002,41,639,6


In [16]:
lens_bins = [-np.inf, 10,20,50,100,200,500,1000,2500,5000,10000,np.inf]
lens_labels = list(range(1,len(lens_bins)))
train['user_len_bin'] = pd.cut(train['user_len'],bins=lens_bins, labels=lens_labels).astype(np.int8)
train[:3]

,target_age,target_is_male,user_id,old_age,user_len,tel_price,user_len_bin
0,1,1,350459,31,178,5,5
1,1,1,188276,35,111,7,5
2,2,0,99002,41,639,6,7


In [17]:
train['target_is_male'].value_counts()

 1    135332
 0    128994
-1      5674
Name: target_is_male, dtype: int64

In [18]:
train['target_age'].value_counts()

 1    87270
 2    77486
 3    42442
 0    33718
 4    23580
 5     5503
-1        1
Name: target_age, dtype: int64

Добавляем столбец для стратификации

In [19]:
train['user_bin'] = train['target_is_male'].replace(-1,0).astype(np.int64) + 10*train['target_age'].replace(-1,1).astype(np.int64) + 100*train['tel_price'].astype(np.int64) + 10000 * train['user_len_bin'].astype(np.int64)
train[:3]

,target_age,target_is_male,user_id,old_age,user_len,tel_price,user_len_bin,user_bin
0,1,1,350459,31,178,5,5,50511
1,1,1,188276,35,111,7,5,50711
2,2,0,99002,41,639,6,7,70620


In [22]:
train['user_bin'].nunique(), train.index

(1183, RangeIndex(start=0, stop=270000, step=1))

In [23]:
user_in_bins = train.groupby('user_bin')['user_id'].transform('count')
p(user_in_bins.index)
user_in_bins

RangeIndex(start=0, stop=270000, step=1)


0          660
1          568
2         1390
3          229
4          448
          ... 
269995     637
269996      21
269997    1612
269998    1528
269999      49
Name: user_id, Length: 270000, dtype: int64

In [24]:
train.loc[user_in_bins<50,'user_bin'] = train['user_bin'].max() + 1
train[:3]

,target_age,target_is_male,user_id,old_age,user_len,tel_price,user_len_bin,user_bin
0,1,1,350459,31,178,5,5,50511
1,1,1,188276,35,111,7,5,50711
2,2,0,99002,41,639,6,7,70620


Разбиваем на 5 фолдов

In [25]:
train = my.add_folds(train, strat_col='user_bin')
train[:5]

,target_age,target_is_male,user_id,old_age,user_len,tel_price,user_len_bin,user_bin,fold
0,1,1,350459,31,178,5,5,50511,3
1,1,1,188276,35,111,7,5,50711,0
2,2,0,99002,41,639,6,7,70620,3
3,1,0,155506,33,22,9,3,30910,2
4,3,0,213873,54,4,4,1,10430,4


In [26]:
train['fold'].value_counts()

3    54000
0    54000
2    54000
4    54000
1    54000
Name: fold, dtype: int64

Определяем бакеты для разных длин последовательностей

In [28]:
buckets = [-np.inf, 20,50,100,200,350,600,1000,1500,2000,2500,np.inf]
bucket_labels = list(range(1,len(buckets)))
train['len_bucket'] = pd.cut(train['user_len'],bins=buckets, labels=bucket_labels).astype(np.int8)
train['len_bucket'].value_counts().sort_index()

1     26667
2     19372
3     19782
4     27419
5     29950
6     36407
7     38590
8     28933
9     16640
10     9624
11    16616
Name: len_bucket, dtype: int64

In [29]:
train[:3]

,target_age,target_is_male,user_id,old_age,user_len,tel_price,user_len_bin,user_bin,fold,len_bucket
0,1,1,350459,31,178,5,5,50511,3,4
1,1,1,188276,35,111,7,5,50711,0,4
2,2,0,99002,41,639,6,7,70620,3,7


In [31]:
train.to_parquet('out/info_targets.pq',index=False)

In [32]:
del train

Создаем вложенный словарь со всей информацией по юзерам из трейна train_val_plts для семплирования потом при тренировке моделей

In [33]:
target_info = pd.read_parquet('out/info_targets.pq')
target_info[:3]

,target_age,target_is_male,user_id,old_age,user_len,tel_price,user_len_bin,user_bin,fold,len_bucket
0,1,1,350459,31,178,5,5,50511,3,4
1,1,1,188276,35,111,7,5,50711,0,4
2,2,0,99002,41,639,6,7,70620,3,7


In [34]:
y_age = target_info[['user_id', 'target_age']].set_index('user_id')['target_age'].to_dict()
y_age[350459], y_age[188276]

(1, 1)

In [36]:
len(y_age)

270000

In [35]:
X_train_val[:2]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,pos
0,29,531,28,156,117490,3,1,4,2021-06-16 02:00:00,2,0.083,0,130
1,29,531,28,156,111474,3,1,4,2021-06-16 02:00:00,2,0.083,0,129


In [38]:
train_val_plts = {}

for user, user_data in tqdm(X_train_val.groupby('user_id'),total=X_train_val['user_id'].nunique()):

    train_val_plts[user] = my.get_user_seqs(user_data)
    train_val_plts[user]['y'] = y_age[user]

len(train_val_plts)

  0%|          | 0/270000 [00:00<?, ?it/s]

270000

In [39]:
my.save_pickle('out/train_val_plts.pik',train_val_plts)

In [40]:
train_val_plts[0]

{'region_name': tensor([29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29], dtype=torch.int32),
 'city_name': tensor([29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29

In [ ]:
del X_train_val
gc.collect()

Создаем вложенный словарь со всей информацией по юзерам из теста - test_plts для семплирования потом при предсказании

In [2]:
X_test = pd.read_parquet('out/pre/X_test_1.pq')
p(X_test.shape)
X_test[:3]

(109567390, 13)


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,pos
0,30,239,14,274,59367,3,1,4,2021-06-16 01:00:00,1,0.167,6,455
1,30,239,14,274,160868,3,1,4,2021-06-16 01:00:00,1,0.250,6,454
2,30,239,14,274,160873,3,1,4,2021-06-16 01:00:00,1,0.417,6,453


In [3]:
test_plts = []

for user_id, user_data in tqdm(X_test.groupby('user_id'),total=X_test['user_id'].nunique()):
    user_dict = my.get_user_seqs(user_data)
    user_dict['user_id'] = user_id
    test_plts.append(user_dict)
    
test_plts.sort(key=lambda x:len(x['url_host']))   
len(test_plts)

  0%|          | 0/144724 [00:00<?, ?it/s]

144724

In [ ]:
from ptls.frames.inference_module import InferenceModule

In [4]:
test_ids = []
for i in range(len(test_plts)):
    test_ids.append(test_plts[i]['user_id'])
    del test_plts[i]['user_id']

In [7]:
test_plts[0]

{'region_name': tensor([55], dtype=torch.int32),
 'city_name': tensor([55], dtype=torch.int32),
 'cpe_manufacturer_name': 34,
 'cpe_model_name': 524,
 'cpe_type_cd': 3,
 'cpe_model_os_type': 1,
 'price': 4,
 'event_time': tensor([1625792400]),
 'dayofweek': tensor([5], dtype=torch.int32),
 'diff_time': tensor([0.]),
 'part_of_day': tensor([1], dtype=torch.int32),
 'url_host': tensor([59367], dtype=torch.int32),
 'request_cnt': tensor([0.0833])}

In [8]:
test_plts = {'test_ids':test_ids,'test_plts':test_plts}

In [11]:
len(test_plts['test_ids']), len(test_plts['test_plts'])

(144724, 144724)

In [12]:
my.save_pickle('out/test_plts.pik',test_plts)

Распределение длин последовательностей в трейне

In [14]:
for i in [10,20,50,100,200,500,1000,5000,10000]:
    print(f'>{i}: {len(q[q>i])} | <{i}: {len(q[q<i])}')

>10: 253257 <10: 15538
>20: 243333 <20: 25800
>50: 223961 <50: 45517
>100: 204179 <100: 65513
>200: 176760 <200: 92996
>500: 123443 <500: 146421
>1000: 71813 <1000: 198117
>5000: 2415 <5000: 267584
>10000: 128 <10000: 269872


Распределение длин последовательностей в тесте

In [20]:
for i in [10,20,50,100,200,500,1000,5000,10000]:
    print(f'>{i}: {len(q[q>i])} | <{i}: {len(q[q<i])}')

144724
>10: 135608 | <10: 8494
>20: 130377 | <20: 13881
>50: 120011 | <50: 24456
>100: 109178 | <100: 35367
>200: 94447 | <200: 50164
>500: 65742 | <500: 78905
>1000: 37959 | <1000: 106719
>5000: 1271 | <5000: 143453
>10000: 84 | <10000: 144640
